In [1]:
import pandas as pd
import numpy as np
import talib as ta
from pycoingecko import CoinGeckoAPI
from datetime import datetime as dt, timedelta as td

In [2]:
config = {
    'id': 'cardano',
    'vs_curr': 'usd',
    'range': 24, # max 24
    'save_csv_path': 'crypto_ta_ada.csv'
}

In [3]:
now = dt.strptime(dt.now().strftime("%Y-%m-%d %H:00:00"),"%Y-%m-%d %H:00:00")
timestamp_list = [{'from': dt.timestamp(now - td(days=(x+1)*90)), 'to': dt.timestamp(now - td(days=x*90))} for x in reversed(range(config['range']))]

print(timestamp_list)

[{'from': 1443873600.0, 'to': 1451653200.0}, {'from': 1451653200.0, 'to': 1459425600.0}, {'from': 1459425600.0, 'to': 1467201600.0}, {'from': 1467201600.0, 'to': 1474977600.0}, {'from': 1474977600.0, 'to': 1482757200.0}, {'from': 1482757200.0, 'to': 1490529600.0}, {'from': 1490529600.0, 'to': 1498305600.0}, {'from': 1498305600.0, 'to': 1506081600.0}, {'from': 1506081600.0, 'to': 1513861200.0}, {'from': 1513861200.0, 'to': 1521637200.0}, {'from': 1521637200.0, 'to': 1529409600.0}, {'from': 1529409600.0, 'to': 1537185600.0}, {'from': 1537185600.0, 'to': 1544965200.0}, {'from': 1544965200.0, 'to': 1552741200.0}, {'from': 1552741200.0, 'to': 1560513600.0}, {'from': 1560513600.0, 'to': 1568289600.0}, {'from': 1568289600.0, 'to': 1576069200.0}, {'from': 1576069200.0, 'to': 1583845200.0}, {'from': 1583845200.0, 'to': 1591617600.0}, {'from': 1591617600.0, 'to': 1599393600.0}, {'from': 1599393600.0, 'to': 1607173200.0}, {'from': 1607173200.0, 'to': 1614949200.0}, {'from': 1614949200.0, 'to': 16

In [4]:
cg = CoinGeckoAPI()
temp_closes = []
temp_volumes = []

for timestamp in timestamp_list:
    mk_chart = cg.get_coin_market_chart_range_by_id(id=config['id'], vs_currency=config['vs_curr'], from_timestamp=timestamp['from'], to_timestamp=timestamp['to'])
    for i in range(len(mk_chart['prices'])):
        temp_closes.append(mk_chart['prices'][i][1])
        temp_volumes.append(mk_chart['total_volumes'][i][1])

closes = np.array(temp_closes)
volumes = np.array(temp_volumes)

print(closes)
print(volumes)

[0.02684535 0.02682995 0.03029961 ... 2.783313   2.83705945 2.82530718]
[2.35167812e+06 2.81515597e+06 8.88347308e+06 ... 3.88282632e+09
 4.00191086e+09 4.15833457e+09]


In [5]:
sma_7 = ta.ROC(ta.SMA(closes, 7), timeperiod=1)
sma_25 = ta.ROC(ta.SMA(closes, 25), timeperiod=1)
sma_99 = ta.ROC(ta.SMA(closes, 99), timeperiod=1)
sma_200 = ta.ROC(ta.SMA(closes, 200), timeperiod=1)
ema_9 = ta.ROC(ta.EMA(closes, 9), timeperiod=1)
ema_26 = ta.ROC(ta.EMA(closes, 26), timeperiod=1)
sma_vol = ta.ROC(ta.SMA(volumes, 20), timeperiod=1)
bbands_up, bbands_mid, bbands_low = ta.BBANDS(closes, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
bbands_up = ta.ROC(bbands_up, timeperiod=1)
bbands_mid = ta.ROC(bbands_mid, timeperiod=1)
bbands_low = ta.ROC(bbands_low, timeperiod=1)
rsi = ta.RSI(closes, 14)
cmo = ta.CMO(closes, timeperiod=14)
macd, macdsignal, macdhist = ta.MACD(closes, fastperiod=12, slowperiod=26, signalperiod=9)
ppo = ta.PPO(closes, fastperiod=12, slowperiod=26, matype=0)
roc = ta.ROC(closes, timeperiod=1)
linearreg = ta.ROC(ta.LINEARREG(closes, timeperiod=14), timeperiod=1)
tsf = ta.ROC(ta.TSF(closes, timeperiod=14), timeperiod=1)

In [6]:
class_data = []

for n in ta.ROC(closes, timeperiod=6):
    if n < -3:
        class_data.append('V_LOW')
    elif -3 <= n <= 0:
        class_data.append('LOW')
    elif 0 < n <= 3:
        class_data.append('HIGH')
    else:
        class_data.append('V_HIGH')

In [7]:
dt = pd.DataFrame({
    'sma_7': sma_7[201:-6],
    'sma_25': sma_25[201:-6],
    'sma_99': sma_99[201:-6],
    'sma_200': sma_200[201:-6],
    'ema_9': ema_9[201:-6],
    'ema_26': ema_26[201:-6],
    'sma_vol': sma_vol[201:-6],
    'bbands_up': bbands_up[201:-6],
    'bbands_mid': bbands_mid[201:-6],
    'bbands_low': bbands_low[201:-6],
    'rsi': rsi[201:-6],
    'cmo': cmo[201:-6],
    'macd': macd[201:-6],
    'macdsignal': macdsignal[201:-6],
    'macdhist': macdhist[201:-6],
    'ppo': ppo[201:-6],
    'roc': roc[201:-6],
    'linearreg': linearreg[201:-6],
    'tsf': tsf[201:-6],
    'trend': np.array(class_data[207::])
})

dt.describe()
print(dt)

          sma_7    sma_25    sma_99   sma_200     ema_9    ema_26   sma_vol  \
0     -0.554958  2.531938 -0.999672  0.526531  0.037076  1.278710 -8.327822   
1     -0.275160  1.636650 -0.925994  0.494158 -0.816688  0.806651 -5.955354   
2     -1.378247  1.629559 -0.963406  0.474333 -1.379964  0.437072 -1.792471   
3     -2.164224  1.519429 -0.678862  0.463992 -1.461086  0.261597 -2.650524   
4     -3.171230  0.954201 -0.751381  0.431485 -2.418561 -0.259445 -3.753348   
...         ...       ...       ...       ...       ...       ...       ...   
21950 -0.061417  0.008598 -0.047778 -0.031060 -0.270287 -0.128994 -0.102366   
21951 -0.119051  0.052130 -0.061668 -0.026206 -0.065570 -0.063877  0.282723   
21952 -0.072742  0.033127 -0.048956 -0.022746 -0.024800 -0.048982  0.309924   
21953 -0.038156 -0.031774 -0.043034 -0.028179 -0.046028 -0.055024 -0.633501   
21954  0.023975 -0.039733 -0.027076 -0.029524  0.044772 -0.020902 -0.658472   

       bbands_up  bbands_mid  bbands_low        rsi

In [8]:
dt.to_csv(config['save_csv_path'], index=False)